# P10: Global GPU Supply Chain - Multiagent Pattern


<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together.

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**P10: Global GPU Supply Chain** - Large-scale industrial project planning with procurement, infrastructure dependencies, and cost-risk tradeoffs.


In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


In [ ]:
import sys
print("Python version:", sys.version)


In [ ]:
%pip install openai python-dotenv graphviz colorama


In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")


In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)


## The Agent Class


First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.


In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")


You can also associate tools with the agent. Let's create a tool for writing some string into a file.


In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")


## The Crew for P10: Global GPU Supply Chain


In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Supplier & Component Setup Agent ---- #
    SC_Agent = Agent(
        name="Supplier & Component Setup Agent",
        backstory="You define suppliers, components, facilities, and their capabilities in the global supply chain.",
        task_description="Set up supply chain structure: suppliers (supplier1: Asia, capacity 1000, lead time 30 days; supplier2: Europe, capacity 800, lead time 45 days), components (GPU chips, memory, cooling, power supply, PCB), and facilities (assembly: USA, capacity 500; testing: USA, capacity 300).",
        task_expected_output="Structured supply chain data: 1) Suppliers with locations, capacities, and lead times. 2) Components required. 3) Facilities with capacities and locations. 4) Budget limit: $1,000,000."
    )
    
    # ---- Procurement Planning Agent ---- #
    PP_Agent = Agent(
        name="Procurement Planning Agent",
        backstory="You plan component procurement from suppliers to meet delivery deadlines while minimizing costs.",
        task_description="Create procurement plan: order components from suppliers to meet deadlines (GPU chips: 30 days, memory: 45 days, cooling: 60 days) while staying within budget and respecting supplier capacities.",
        task_expected_output="Procurement plan with component orders, supplier assignments, order quantities, and delivery schedules."
    )
    
    # ---- Infrastructure Planning Agent ---- #
    IP_Agent = Agent(
        name="Infrastructure Planning Agent",
        backstory="You plan infrastructure setup considering dependencies between facilities and processes.",
        task_description="Plan infrastructure setup respecting dependencies: power supply before assembly, testing before packaging. Schedule facility operations within capacity constraints.",
        task_expected_output="Infrastructure plan with facility schedules, dependency sequences, and capacity utilization."
    )
    
    # ---- Cost Optimization Agent ---- #
    CO_Agent = Agent(
        name="Cost Optimization Agent",
        backstory="You optimize total project costs while meeting all constraints and deadlines.",
        task_description="Optimize procurement and operations to minimize total cost while meeting delivery deadlines, supplier capacities, and budget limit of $1,000,000.",
        task_expected_output="Cost-optimized plan with total cost breakdown, supplier selections, and cost-risk tradeoffs."
    )
    
    # ---- Risk Assessment Agent ---- #
    RA_Agent = Agent(
        name="Risk Assessment Agent",
        backstory="You assess supply chain risks including shortages, machine breakdowns, and delays.",
        task_description="Assess risks: potential GPU chip shortages (40% reduction), machine breakdowns at assembly facilities (7 days), and develop mitigation strategies.",
        task_expected_output="Risk assessment with identified risks, impact analysis, and mitigation strategies including alternative suppliers and backup plans."
    )
    
    # ---- Inventory Optimization Agent ---- #
    IO_Agent = Agent(
        name="Inventory Optimization Agent",
        backstory="You optimize inventory levels to balance costs and ensure supply continuity.",
        task_description="Optimize inventory levels for each component to minimize holding costs while ensuring continuous supply and meeting production schedules.",
        task_expected_output="Optimized inventory plan with reorder points, safety stock levels, and inventory cost analysis."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './p10_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    SC_Agent >> PP_Agent >> IP_Agent >> CO_Agent >> RA_Agent >> IO_Agent >> Writer_agent


In [ ]:
crew.run()
